In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets, cluster
import matplotlib.pyplot as plt
import plotly.express as px
# import squarify

import warnings
warnings.filterwarnings('ignore')

import os, pickle
from time import time

from sklearn.decomposition import NMF
import scipy

from proj_util import cal_avg_precision, get_user_product_matrix

FOLDER_PATH = 'drive/My Drive/ML/colab'


In [10]:
train_set = pickle.load(open(os.path.join(FOLDER_PATH,'pickle','train_set.p'),'rb'))
validation_set = pickle.load(open(os.path.join(FOLDER_PATH,'pickle','validation_set.p'),'rb'))

user_mapping = pickle.load(open(os.path.join(FOLDER_PATH,'pickle','user_mapping.p'),'rb'))
product_mapping = pickle.load(open(os.path.join(FOLDER_PATH,'pickle','product_mapping.p'),'rb'))

In [11]:
order_prior_set = pd.concat([train_set,validation_set])
del train_set, validation_set

In [13]:
order_prior_set.user_id = order_prior_set.user_id.map(user_mapping)
order_prior_set.product_id = order_prior_set.product_id.map(product_mapping)

In [14]:
user_product_prior = order_prior_set.groupby(['user_id', 'product_id']).size().reset_index().rename(columns={0: 'quantity'})
qty_max = max(user_product_prior.quantity)

user_product_prior.quantity = user_product_prior.quantity.apply(lambda x: (x-1)/float(qty_max-1))

In [15]:
user_product_prior_coo = get_user_product_matrix(user_product_prior)

In [ ]:
del user_product_train, user_mapping, product_mapping

### CoClust


In [17]:
from coclust.coclustering import *
import coclust
from sklearn.decomposition import NMF

In [25]:
# row: item
# col: user
model = CoclustInfo(n_row_clusters=10,n_col_clusters=40,random_state=421)
model.fit(user_product_prior_coo)

CoclustInfo(init=None, max_iter=20, n_col_clusters=40, n_init=1,
            n_row_clusters=10, random_state=421, tol=1e-09)

In [29]:
pickle.dump(model,open(os.path.join(FOLDER_PATH,'pickle','coclust-model-prior.p'),'wb'))

In [ ]:
for item_cluster in range(10):
    for user_cluster in range(40):

        user_item_cluster = model.get_submatrix(user_product_prior_coo.tocsr(),item_cluster,user_cluster)

        nmf = NMF(n_components=120, random_state=421, alpha=0.001) 
        nmf.fit(user_item_cluster)

        pickle.dump(nmf, open(os.path.join(FOLDER_PATH,'pickle','coclust-nmf-prior',f'{item_cluster}-{user_cluster}-nmf.p'),'wb'))
        print(f'{item_cluster}-{user_cluster}-nmf')
        del nmf, user_item_cluster